In [1]:
import os 
import jax
import tax
import brax
import jax.numpy as jnp
import functools
import matplotlib as mpl
import matplotlib.pyplot as plt

from jax import jit 
from jax import vmap
from brax import envs
from brax.envs.ant import Ant
from brax.training import ppo, sac
from brax.io import html

tax.set_platform('cpu')

In [2]:
rng = jax.random.PRNGKey(42)
env = Ant()

In [3]:
@jit
def step(carry, xs):
    rng, state = carry
    rng, rng_action, rng_reset = jax.random.split(rng, 3)
    
    action = jax.random.uniform(rng_action, shape=(env.action_size,), 
                               minval=-1.0, maxval=1.0)
    new_state = env.step(state, action)    
    done = new_state.done
    
    """
    Seems to be quite slow. => Add Options.
    =======================
    """
    # Do nothing if the episode is not terminated.
    # else reset the state (using the `rng_reset` seed).
    #new_state = \
    #    jax.lax.cond(done, lambda x: env.reset(x[0]), lambda x: x[1], (rng_reset, new_state))
    
    carry = [rng, new_state]
    info = {
        'reward': new_state.reward,
        'observation': state.obs,
        'observation_next': new_state.obs,
        'terminal': 1.0 - new_state.done,
        'steps': new_state.steps,
    }
    return carry, info

In [4]:
@jit
def episode(rng):
    state = env.reset(rng)
    init, xs = [rng, state], jnp.arange(1000)
    _, info = jax.lax.scan(step, init, xs)
    return info

In [5]:
%%time
state = env.reset(rng)
init, xs = [rng, state], jnp.arange(10)
_, info = jax.lax.scan(step, init, xs)

CPU times: user 31.9 s, sys: 80.1 ms, total: 31.9 s
Wall time: 32 s


In [6]:
episode(rng)['terminal']

DeviceArray([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
             1., 1., 1., 1., 1., 1., 1., 1., 1., 1.

In [7]:
# Batched Version.
# ================

brng  = jax.random.split(rng, 8)
binfo = jit(vmap(episode))(brng)